<a href="https://colab.research.google.com/github/ValentinaEmili/Sign_language/blob/main/DataAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# mount google drive on colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import numpy as np
import os

In [5]:
js_100 = pd.read_json("/content/drive/MyDrive/NLP/WLASL100.json")
folder = "/content/drive/MyDrive/NLP/dataset/subset_100/"
original_folder = "/content/drive/MyDrive/NLP/dataset/"

training_folder = folder + "train/"
validation_folder = folder + "val/"
test_folder = folder + "test/"

training_video = training_folder + "video/"
validation_video = validation_folder + "video/"
test_video = test_folder + "video/"

training_images = training_folder + "images/"
validation_images = validation_folder + "images/"
test_images = test_folder + "images/"

os.makedirs(training_video, exist_ok=True)
os.makedirs(validation_video, exist_ok=True)
os.makedirs(test_video, exist_ok=True)

os.makedirs(training_images, exist_ok=True)
os.makedirs(validation_images, exist_ok=True)
os.makedirs(test_images, exist_ok=True)


In [ ]:
video_ids = []
for video in os.listdir(original_folder + "train/video/"):
  video_ids.append(int(video[:-4].split("_")[1]))
for video in os.listdir(original_folder + "val/video/"):
  video_ids.append(int(video[:-4].split("_")[1]))
for video in os.listdir(original_folder + "test/video/"):
  video_ids.append(int(video[:-4].split("_")[1]))

new_id = max(video_ids)+1

In [36]:
new_id = 70380

every video id >= 70380 is generated by some other image adding some noise or transformation

In [21]:
def add_gaussian_noise(seq, mean=0, std=0.01):
  noise = np.random.normal(mean, std, seq.shape)
  return seq + noise

def jitter_joints(seq, jitter_scale=0.02):
  return seq + np.random.uniform(-jitter_scale, jitter_scale, seq.shape)

def random_dropout(seq, drop_prob=0.1):
  mask = (np.random.rand(*seq.shape) < drop_prob) /drop_prob
  return seq * mask

In [10]:
all_labels = [image.split("_")[0] for image in os.listdir(training_images)]
set_labels = set(all_labels)
label_counts = {label: all_labels.count(label) for label in set_labels}

original dataset:
- training set: 914
- validation set: 211
- test set: 189

In [ ]:
for image in os.listdir(training_images):
  word, _ = image.split("_")
  if label_counts[word] < 8:
    np_array = np.load(os.path.join(training_images, image))

    gauss_array = add_gaussian_noise(np_array)
    gauss_image = f"{word}_{new_id}.npy"
    new_id += 1
    np.save(os.path.join(training_images, gauss_image), gauss_array)

    jitter_points_array =  jitter_joints(np_array)
    jitter_image = f"{word}_{new_id}.npy"
    new_id += 1
    np.save(os.path.join(training_images, jitter_image), jitter_points_array)

    drop_frames_array = random_dropout(np_array)
    drop_frames_image = f"{word}_{new_id}.npy"
    new_id += 1
    np.save(os.path.join(training_images, drop_frames_image), drop_frames_array)

after:
- training set: 1289
- validation set: 211
- test set: 189